In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
output_data_dir = '/group/jug/ashesh/naturemethods/similarityExperiment'
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="2"


In [ ]:
# there are two environments(debug and prod). From where you want to fetch the code and data? 
DEBUG=False

In [ ]:
%run ../nb_core/root_dirs.ipynb
setup_syspath_disentangle(DEBUG)
%run ../nb_core/disentangle_imports.ipynb

In [ ]:
ckpt_dir = "/group/jug/ashesh/training/disentangle/2502/D31-M3-S0-L8/20"
# 211/D3-M3-S0-L0/0
# 2210/D3-M3-S0-L0/128
# 2210/D3-M3-S0-L0/129

In [ ]:
# !ls /home/ubuntu/ashesh/training/disentangle/2209/D3-M9-S0-L0/1

In [ ]:
from disentangle.data_loader.multifile_raw_dloader import SubDsetType
from disentangle.data_loader.patch_index_manager import TilingMode

image_size_for_grid_centers = 32
mmse_count = 1
custom_image_size = None
data_t_list = None #[0,1,2,3]
tiling_mode = TilingMode.ShiftBoundary


batch_size = 8
num_workers = 4
COMPUTE_LOSS = False
use_deterministic_grid = None
threshold = None # 0.02
compute_kl_loss = False
evaluate_train = False# inspect training performance
eval_datasplit_type = DataSplitType.Test
val_repeat_factor = None
psnr_type = 'range_invariant' #'simple', 'range_invariant'

In [ ]:
%run ../nb_core/config_loader.ipynb

In [ ]:
from disentangle.core.sampler_type import SamplerType
from disentangle.core.loss_type import LossType
# from disentangle.core.lowres_merge_type import LowresMergeType
from disentangle.data_loader.multifile_raw_dloader import SubDsetType

with config.unlocked():
    config.model.skip_nboundary_pixels_from_loss = None
    if config.model.model_type == ModelType.UNet and 'n_levels' not in config.model:
        config.model.n_levels = 4
    if config.data.sampler_type == SamplerType.NeighborSampler:
        config.data.sampler_type = SamplerType.DefaultSampler
        config.loss.loss_type = LossType.Elbo
        config.data.grid_size = config.data.image_size
    if 'ch1_fpath_list' in config.data:
        config.data.ch1_fpath_list = config.data.ch1_fpath_list[:1]
        config.data.mix_fpath_list = config.data.mix_fpath_list[:1]
    if config.data.data_type == DataType.Pavia2VanillaSplitting:
        if 'channel_2_downscale_factor' not in config.data:
            config.data.channel_2_downscale_factor = 1
    if config.model.model_type == ModelType.UNet and 'init_channel_count' not in config.model:
        config.model.init_channel_count = 64
    
    if 'skip_receptive_field_loss_tokens' not in config.loss:
        config.loss.skip_receptive_field_loss_tokens = []
    
    if config.data.data_type == DataType.HTIba1Ki67:
        config.data.subdset_type = SubDsetType.Iba1Ki64
        config.data.empty_patch_replacement_enabled = False
    
    if 'lowres_merge_type' not in config.model.encoder:
        config.model.encoder.lowres_merge_type = 0
    
    if config.data.data_type == DataType.TwoDset:
        config.model.model_type = ModelType.LadderVae
        for key in config.data.dset1:
            config.data[key] = config.data.dset1[key]
    if config.data.data_type == DataType.TavernaSox2GolgiV2:
        config.data.channel_1 = '555-647'
        config.data.channel_2 = '555-647'


In [ ]:
dtype = config.data.raw_data_type
dtype

In [ ]:
if dtype in [DataType.CustomSinosoid, DataType.CustomSinosoidThreeCurve]:
    data_dir = f'{DATA_ROOT}/sinosoid_without_test/sinosoid/'
elif dtype == DataType.OptiMEM100_014:
    data_dir = f'{DATA_ROOT}/microscopy/'
elif dtype == DataType.Prevedel_EMBL:
    data_dir = f'{DATA_ROOT}/Prevedel_EMBL/PKG_3P_dualcolor_stacks/NoAverage_NoRegistration/'
elif dtype == DataType.AllenCellMito:
    data_dir = f'{DATA_ROOT}/allencell/2017_03_08_Struct_First_Pass_Seg/AICS-11/'
elif dtype == DataType.SeparateTiffData:
    data_dir = f'{DATA_ROOT}/ventura_gigascience'
elif dtype == DataType.SemiSupBloodVesselsEMBL:
    data_dir = f'{DATA_ROOT}/EMBL_halfsupervised/Demixing_3P'
elif dtype == DataType.Pavia2VanillaSplitting:
    data_dir = f'{DATA_ROOT}/pavia2'
# elif dtype == DataType.ExpansionMicroscopyMitoTub:
    # data_dir = f'{DATA_ROOT}/expansion_microscopy_Nick/'
elif dtype == DataType.ShroffMitoEr:
    data_dir = f'{DATA_ROOT}/shrofflab/'
elif dtype == DataType.HTIba1Ki67:
    data_dir = f'{DATA_ROOT}/Stefania/20230327_Ki67_and_Iba1_trainingdata/'
elif dtype == DataType.BioSR_MRC:
    data_dir = f'{DATA_ROOT}/BioSR/'
elif dtype == DataType.ExpMicroscopyV2:
    data_dir = f'{DATA_ROOT}/expansion_microscopy_v2/datafiles/'
elif dtype == DataType.TavernaSox2GolgiV2:
    data_dir = f'{DATA_ROOT}/TavernaSox2Golgi/acquisition2/'
elif dtype == DataType.Pavia3SeqData:
    data_dir = f'{DATA_ROOT}/pavia3_sequential/'
elif dtype == DataType.NicolaData:
    data_dir = f'{DATA_ROOT}/nikola_data/20240531/'
elif dtype == DataType.Dao3ChannelWithInput:
    data_dir = f'{DATA_ROOT}/Dao4Channel/'
elif dtype == DataType.Dao3Channel:
    data_dir = f'{DATA_ROOT}/Dao3Channel/'
elif dtype == DataType.SilvioLabCSHLData:
    data_dir = f'{DATA_ROOT}/svilen_cshl2024/'
elif dtype == DataType.ExpMicroscopyV3:
    data_dir = f'{DATA_ROOT}/expansion_microscopy_v4/405_NHS_488BODIPY/'
elif dtype == DataType.Elisa3DData:
    data_dir = f'{DATA_ROOT}/Elisa3D/'

In [ ]:
%run ../nb_core/disentangle_setup.ipynb

In [ ]:
len(val_dset._data_arr)

In [ ]:
plt.imshow(val_dset._data_arr[0][1])

In [ ]:
data1 = val_dset._data_arr[0].copy()
data2 = val_dset._data_arr[1][:,:data1.shape[1], :data1.shape[2]].copy()

In [ ]:
data = np.concatenate([data1[...,None], data2[::-1,...,None]], axis=-1)

In [ ]:
import numpy as np

class YourDataset:
    def __init__(self, data, mean_dict, std_dict, patch_size=64) -> None:
        self.data = data
        self.mean_dict = mean_dict
        self.std_dict = std_dict
        self.patch_size = patch_size
        
    def patch_location(self, index:int):
        # it just ignores the index and returns a random location
        n_idx = np.random.randint(0,len(self.data))
        h = np.random.randint(0, self.data.shape[1]-self.patch_size)
        w = np.random.randint(0, self.data.shape[2]-self.patch_size)
        return (n_idx, h, w)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        n_idx, h, w = self.patch_location(index)
        # return the patch at the location (patch_size, patch_size)
        target = self.data[n_idx, h:h+self.patch_size, w:w+self.patch_size]
        target = target.transpose(2,0,1)
        inp = np.sum(target, axis=0, keepdims=True)
        inp = (inp - self.mean_dict['input'].squeeze())/self.std_dict['input'].squeeze()
        target = (target - self.mean_dict['target'].squeeze().reshape(-1,1,1))/self.std_dict['target'].squeeze().reshape(-1,1,1)
        return inp, target

In [ ]:
dset = YourDataset(data, val_dset._data_mean, val_dset._data_std)

In [ ]:
inp, tar = dset[4]
_,ax = plt.subplots(figsize=(12,4),ncols=3)
ax[0].imshow(inp[0], cmap='gray')
ax[1].imshow(tar[0], cmap='gray')
ax[2].imshow(tar[1], cmap='gray')

In [ ]:
import torch
with torch.no_grad():

    inp_torch = torch.tensor(inp).float().unsqueeze(0).cuda()
    
    out,_ = model(inp_torch)
    out = out.cpu().numpy().squeeze()
    out = out[:2]

In [ ]:
_,ax = plt.subplots(figsize=(8,4),ncols=2)
ax[0].imshow(out[0], cmap='gray')
ax[1].imshow(out[1], cmap='gray')

In [ ]:
from predtiler import get_tiling_dataset, get_tile_manager, stitch_predictions

patch_size = 64
tile_size = 32
data_shape = data.shape[:-1]
manager = get_tile_manager(data_shape=data_shape, tile_shape=(1,tile_size,tile_size), 
                               patch_shape=(1,patch_size,patch_size))
    
dset_class = get_tiling_dataset(YourDataset, manager)
dset = dset_class(data, val_dset._data_mean, val_dset._data_std, patch_size=patch_size)


In [ ]:
from tqdm import tqdm
predictions = []

dloader = torch.utils.data.DataLoader(dset, batch_size=16, num_workers=2)
mmse_count = 10
for batch in tqdm(dloader):
    inp, _ = batch
    inp = inp.cuda()
    # inp, _ = dset[i]
    # inp = torch.Tensor(inp)[None].cuda()
    mmse_preds = []
    for _ in range(mmse_count):
        pred, _  = model(inp)
        mmse_preds.append(pred.detach().cpu().numpy())
    mmse_preds = np.stack(mmse_preds)
    pred = np.mean(mmse_preds, axis=0)
    predictions.append(pred[:,:2])

predictions = np.concatenate(predictions) # shape: (number_of_patches, C, patch_size, patch_size)
stitched_pred = stitch_predictions(predictions, dset.tile_manager)

In [ ]:
from disentangle.analysis.plot_utils import clean_ax, add_text
from disentangle.scripts.evaluate import avg_range_inv_psnr
save_to_file = True
hs = 0
ws = 0
sz = 400


_,ax = plt.subplots(figsize=(15,3),ncols=5)
idx = 2
ax[0].imshow(data[idx].sum(axis=-1)[hs:hs+sz,ws:ws+sz],cmap='gray')
ax[1].imshow(stitched_pred[idx,hs:hs+sz,ws:ws+sz,0], cmap='gray')
ax[2].imshow(stitched_pred[idx,hs:hs+sz,ws:ws+sz,1], cmap='gray')
ax[3].imshow(data[idx,hs:hs+sz,ws:ws+sz,0], cmap='gray')
ax[4].imshow(data[idx,hs:hs+sz,ws:ws+sz,1], cmap='gray')

ax[0].set_title('Input')
ax[1].set_title('Prediction Ch1')
ax[2].set_title('Prediction Ch2')
ax[3].set_title('Target Ch1')
ax[4].set_title('Target Ch2')

# psnr
psnr1 = avg_range_inv_psnr(data[idx:idx+1,...,0], stitched_pred[idx:idx+1,...,0])[0]
psnr2 = avg_range_inv_psnr(data[idx:idx+1,...,1], stitched_pred[idx:idx+1,...,1])[0]
add_text(ax[1], f'PSNR: {psnr1:.2f}', (sz,sz), alpha=0.6)
add_text(ax[2], f'PSNR: {psnr2:.2f}', (sz,sz), alpha=0.6)

clean_ax(ax)
# reduce size between the subplots

task_token = '/'.join(ckpt_dir.split('/')[-3:])
plt.subplots_adjust(wspace=0.03, hspace=0.03)
fname = f"SimilarityExp-{task_token.replace('/','_')}-Idx{idx}.png"
fpath = os.path.join(output_data_dir, fname)
print(fpath)
if save_to_file:
    plt.savefig(fpath, dpi=150, bbox_inches='tight')



In [ ]:
psnr1

In [ ]:
from disentangle.scripts.evaluate import compute_high_snr_stats
print('Test MMSE-Count', mmse_count, task_token)
_ = compute_high_snr_stats(data, stitched_pred)